<a href="https://colab.research.google.com/github/cuongtm2/machine-learning/blob/master/TF_training_Day_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**LUYỆN TẬP THỰC HÀNH TENSORFLOW**

[DAY 2] TEXT AND IMAGE PROCESSING

In [3]:
!pip install tensorflow==2.3.0
!pip install tensorflow_datasets==3.2.1
!pip install Pillow==7.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import urllib
import zipfile
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

1. Image classification

Bộ dữ liệu Người hay ngựa (Horse or human) được tải về sẵn như dưới

In [5]:
_TRAIN_URL = "https://storage.googleapis.com/download.tensorflow.org/data/horse-or-human.zip"
_TEST_URL = "https://storage.googleapis.com/download.tensorflow.org/data/validation-horse-or-human.zip"
urllib.request.urlretrieve(_TRAIN_URL, 'horse-or-human.zip')
local_zip = 'horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('tmp/horse-or-human/')
zip_ref.close()
urllib.request.urlretrieve(_TEST_URL, 'testdata.zip')
local_zip = 'testdata.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('tmp/testdata/')
zip_ref.close()

Dữ liệu ảnh Người - ngựa đã được lưu vào các thư mục: dir_train, dir_test.
Hãy load dữ liệu này thành dataset và huấn luyện một mô hình phân loại người - ngựa


In [6]:
dir_train = 'tmp/horse-or-human/'
dir_test = 'tmp/testdata/'



In [7]:
from tensorflow.keras.applications.efficientnet import EfficientNetB7, preprocess_input

In [16]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, 
                                   horizontal_flip=True, 
                                   zoom_range=0.2,
                                   shear_range=0.2,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2)
train_generator = train_datagen.flow_from_directory(directory=dir_train, 
                                                    target_size=(224,224),
                                                    class_mode='binary',
                                                    batch_size=32,
                                                    shuffle=True)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_generator = train_datagen.flow_from_directory(directory=dir_test,
                                                  target_size=(224,224), 
                                                  class_mode='binary', 
                                                  batch_size=32,
                                                  shuffle=True)

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [17]:
efn = EfficientNetB7(weights="imagenet", 
                     include_top = False, 
                     input_shape = (224,224,3))
for layer in efn.layers:
    layer.trainable = False

model = tf.keras.Sequential()
model.add(efn)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb7 (Functional)  (None, 7, 7, 2560)        64097687  
_________________________________________________________________
flatten_3 (Flatten)          (None, 125440)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 125441    
Total params: 64,223,128
Trainable params: 125,441
Non-trainable params: 64,097,687
_________________________________________________________________


In [18]:
model.compile(optimizer="adam", 
              loss="binary_crossentropy", 
              metrics ="accuracy")

In [19]:
patient = 4
save_path = '/content/drive/MyDrive/tf_certification/model3.h5'
callbacks_list = [
    EarlyStopping(
        monitor='val_loss', 
        patience=patient, 
        mode='min', 
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor = 'val_loss', 
        factor = 0.5, 
        patience = patient / 2, 
        min_lr=1e-6,
        verbose=1,
        mode='min'
    ),
    ModelCheckpoint(save_path, 
                    monitor="val_accuracy",
                    mode="max",
                    verbose=1,
                    save_best_only=True)
    ]


In [20]:
history = model.fit(train_generator,
                    batch_size=32,
                    epochs=50,
                    validation_data=val_generator,
                    callbacks=callbacks_list)

Epoch 1/50
33/33 [==============================] - ETA: 0s - loss: 0.1859 - accuracy: 0.9640
Epoch 00001: val_accuracy improved from -inf to 0.99609, saving model to /content/drive/MyDrive/tf_certification/model3.h5
33/33 [==============================] - 33s 993ms/step - loss: 0.1859 - accuracy: 0.9640 - val_loss: 0.0997 - val_accuracy: 0.9961
Epoch 2/50
33/33 [==============================] - ETA: 0s - loss: 0.0143 - accuracy: 0.9981
Epoch 00002: val_accuracy improved from 0.99609 to 1.00000, saving model to /content/drive/MyDrive/tf_certification/model3.h5
33/33 [==============================] - 26s 797ms/step - loss: 0.0143 - accuracy: 0.9981 - val_loss: 6.8280e-04 - val_accuracy: 1.0000
Epoch 3/50
33/33 [==============================] - ETA: 0s - loss: 0.0206 - accuracy: 0.9981
Epoch 00003: val_accuracy did not improve from 1.00000
33/33 [==============================] - 23s 709ms/step - loss: 0.0206 - accuracy: 0.9981 - val_loss: 1.6568e-05 - val_accuracy: 1.0000
Epoch 4/50

2. Text classification

Bộ dữ liệu Sarcasm được tải về như dưới, hãy xử lý huấn luyện một mô hình phân loại văn bản là mỉa mai hay không.

In [5]:
import json
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer, one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [6]:
url = 'https://storage.googleapis.com/download.tensorflow.org/data/sarcasm.json'
urllib.request.urlretrieve(url, 'sarcasm.json')

# DO NOT CHANGE THIS CODE OR THE TESTS MAY NOT WORK
vocab_size = 1000
embedding_dim = 16
max_length = 120
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000

sentences = []
labels = []

# YOUR CODE HERE
with open('sarcasm.json', 'r') as f:
    data = json.load(f)

for d in data:
    sentences.append(d['headline'])
    labels.append(d['is_sarcastic'])


In [7]:
len(data)

26709

In [ ]:
# training_size = int(len(data)*0.2)
# train_sentences = sentences[:training_size]
# train_labels = labels[:training_size]
# validation_sentences = sentences[training_size:]
# validation_labels = labels[training_size:]

# # 데이터셋은 numpy array이어야 한다.
# train_labels = np.array(train_labels)
# validation_labels = np.array(validation_labels)

In [34]:
y = np.array(labels)

In [25]:
X_train, X_val, y_train, y_val = train_test_split(sentences, y, test_size=0.20, random_state=42)

In [35]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(sentences)
word_to_idx = tokenizer.word_index

In [36]:
train_sequences = tokenizer.texts_to_sequences(X_train)
validation_sequences = tokenizer.texts_to_sequences(X_val)

In [37]:
train_padded = pad_sequences(train_sequences, truncating=trunc_type, padding=padding_type, maxlen=max_length)
validation_padded = pad_sequences(validation_sequences, padding=padding_type, maxlen=max_length)

In [38]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [39]:
patient = 4
save_path = '/content/drive/MyDrive/tf_certification/model4.h5'
callbacks_list = [
    EarlyStopping(
        monitor='val_loss', 
        patience=patient, 
        mode='min', 
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor = 'val_loss', 
        factor = 0.5, 
        patience = patient / 2, 
        min_lr=1e-6,
        verbose=1,
        mode='min'
    ),
    ModelCheckpoint(save_path, 
                    monitor="val_accuracy",
                    mode="max",
                    verbose=1,
                    save_best_only=True)
    ]


In [40]:
train_padded.shape , y_train.shape, validation_padded.shape,y_val.shape

((21367, 120), (21367,), (5342, 120), (5342,))

In [32]:
history = model.fit(x= train_padded, y =y_train,
                    batch_size=32,
                    epochs=50,
                    validation_data=(validation_padded, y_val),
                    callbacks=callbacks_list)

Epoch 1/50
667/668 [============================>.] - ETA: 0s - loss: 0.4837 - accuracy: 0.7569
Epoch 00001: val_accuracy improved from -inf to 0.82216, saving model to /content/drive/MyDrive/tf_certification/model4.h5
668/668 [==============================] - 9s 13ms/step - loss: 0.4837 - accuracy: 0.7570 - val_loss: 0.3792 - val_accuracy: 0.8222
Epoch 2/50
667/668 [============================>.] - ETA: 0s - loss: 0.3681 - accuracy: 0.8363
Epoch 00002: val_accuracy improved from 0.82216 to 0.83620, saving model to /content/drive/MyDrive/tf_certification/model4.h5
668/668 [==============================] - 8s 12ms/step - loss: 0.3681 - accuracy: 0.8362 - val_loss: 0.3690 - val_accuracy: 0.8362
Epoch 3/50
667/668 [============================>.] - ETA: 0s - loss: 0.3366 - accuracy: 0.8515
Epoch 00003: val_accuracy did not improve from 0.83620
668/668 [==============================] - 12s 18ms/step - loss: 0.3366 - accuracy: 0.8515 - val_loss: 0.3695 - val_accuracy: 0.8353
Epoch 4/50


In [33]:
vocab_size = 10000

In [42]:
patient = 4
save_path = '/content/drive/MyDrive/tf_certification/model4_1.h5'
callbacks_list = [
    EarlyStopping(
        monitor='val_loss', 
        patience=patient, 
        mode='min', 
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor = 'val_loss', 
        factor = 0.5, 
        patience = patient / 2, 
        min_lr=1e-6,
        verbose=1,
        mode='min'
    ),
    ModelCheckpoint(save_path, 
                    monitor="val_accuracy",
                    mode="max",
                    verbose=1,
                    save_best_only=True)
    ]


In [43]:
history = model.fit(x= train_padded, y =y_train,
                    batch_size=32,
                    epochs=50,
                    validation_data=(validation_padded, y_val),
                    callbacks=callbacks_list)

Epoch 1/50
666/668 [============================>.] - ETA: 0s - loss: 0.4575 - accuracy: 0.7841
Epoch 00001: val_accuracy improved from -inf to 0.85548, saving model to /content/drive/MyDrive/tf_certification/model4_1.h5
668/668 [==============================] - 10s 16ms/step - loss: 0.4574 - accuracy: 0.7842 - val_loss: 0.3297 - val_accuracy: 0.8555
Epoch 2/50
668/668 [==============================] - ETA: 0s - loss: 0.2550 - accuracy: 0.9034
Epoch 00002: val_accuracy improved from 0.85548 to 0.86148, saving model to /content/drive/MyDrive/tf_certification/model4_1.h5
668/668 [==============================] - 9s 14ms/step - loss: 0.2550 - accuracy: 0.9034 - val_loss: 0.3297 - val_accuracy: 0.8615
Epoch 3/50
668/668 [==============================] - ETA: 0s - loss: 0.1852 - accuracy: 0.9351
Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 00003: val_accuracy did not improve from 0.86148
668/668 [==============================] - 9s 13ms/step - 